## Create a calibration using pyFAI

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyFAI.gui.jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
import pathlib
from tqdm import tqdm
import logging
import math
import warnings
import os
import pyFAI.azimuthalIntegrator
from pyFAI.gui import jupyter
figure_size = 10

## Calibrate the detector

In [ ]:
def calibrate(input_calibrant_path: str, beam_centre_x: float, beam_centre_y: float, sample_detector_distance: float, 
              wl: float, pixel_x: float, pixel_y: float, calibrant_type: str, num_calibration_rings: str, 
              output_calibration_path: str):
    
    frame = fabio.open(input_calibrant_path).data
    dimensions = np.shape(frame)
    print(f"Size of the detector: \n {dimensions} \n")
    figure = plt.figure(figsize=(figure_size, figure_size))
    ax = plt.gca()
    pyFAI.gui.jupyter.display(frame, ax=ax)
    
    detector = pyFAI.detectors.Detector(pixel1=pixel_x, pixel2=pixel_y) # DESY 2021
    print(f"Definition of the detector: \n {detector} \n")
    calibrant = CALIBRANT_FACTORY(calibrant_type)
    calibrant.wavelength = wl
    print(f"Definition of the calibrant: \n {calibrant} \n")
    
    initial_geometry = pyFAI.geometry.Geometry(detector=detector, wavelength=wl)
    initial_geometry.setFit2D(sample_detector_distance, beam_centre_x, beam_centre_y)
    print(f"Initial guessed detector geometry: \n {initial_geometry} \n")
    
    # use the SingleGeometry object to perform automatic ring extraction and calibration
    sg = SingleGeometry("demo", frame, calibrant=calibrant, detector=detector, geometry=initial_geometry)
    sg.extract_cp(max_rings=num_calibration_rings)
    
    # figure showing initial fit
    figure = plt.figure(figsize=(figure_size, figure_size))
    ax = plt.gca()
    pyFAI.gui.jupyter.display(sg=sg, ax=ax)
    
    # refine geometry with fixed wavelength
    sg.geometry_refinement.refine2(fix=["wavelength"])
    
    # figure showing refined fit
    figure = plt.figure(figsize=(figure_size, figure_size))
    ax = plt.gca()
    pyFAI.gui.jupyter.display(sg=sg, ax=ax)
    
    print(f"Final calibration parameters... \n")
    # delete the calibration if it already exists
    # pathlib.Path(calibration_path).unlink(missing_ok=True) # only works for 3.8+
    try:
        pathlib.Path(output_calibration_path).unlink()
    except FileNotFoundError:
        pass

    # save the geometry obtained
    sg.geometry_refinement.save(output_calibration_path)
    with open(output_calibration_path) as f:
        print(f.read())
        
    print(f"Calibration .poni file written to: {output_calibration_path}")

Define geometry components such as beam centre and detector distance.

In [ ]:
# Diamond 2017
input_calibrant_file = "../../../SXRD_raw_data/diamond_2017/rawdata_calibration/67098/pixi_00001.tif"

beam_centre_x = 712.136 # x-coordinate of the beam-center in pixels
beam_centre_y = 727.864 # y-coordinate of the beam-center in pixels
sample_detector_distance = 1194.046 # This is the distance in mm (unit used by Fit2d)
wl = 1.393000e-11 # Beam wavelength

pixel_x = 0.000296
pixel_y = 0.000296

calibrant_type = "CeO2"
num_calibration_rings = 14

output_calibration_path = "../../../SXRD_analysis/diamond_2017/calibration-pyFAI/Diamond_2017_CeO2_1200mm_pyFAI.poni"

In [ ]:
# DESY 2021
input_calibrant_file = "../../../SXRD_raw_data/desy_2021/diffraction_images/LaB6/LaB6_5mm_Dilatometer-00003.tif"

beam_centre_x = 1034.850 # x-coordinate of the beam-center in pixels
beam_centre_y = 2048 - 1020.259 # y-coordinate of the beam-center in pixels
sample_detector_distance = 1554.0496 # This is the distance in mm (unit used by Fit2d)
wl = 1.240000e-11 # Beam wavelength

pixel_x = 0.000200
pixel_y = 0.000200

calibrant_type = "LaB6"
num_calibration_rings = 14

output_calibration_path = "../../../SXRD_analysis/desy_2021/calibration-pyFAI/DESY_2021_LaB6_1554mm_Dilatometer_pyFAI.poni"

In [ ]:
# Diamond 2021
input_calibrant_file = "../../../SXRD_raw_data/diamond_2021/rawdata_tiffs/103852_tiffs/00001_00000.tiff"

beam_centre_x = 753.533 # x-coordinate of the beam-center in pixels
beam_centre_y = 1679 - 998.707 # y-coordinate of the beam-center in pixels
sample_detector_distance = 737.2873 # This is the distance in mm (unit used by Fit2d)
wl = 1.242000e-11 # Beam wavelength

pixel_x = 0.000172
pixel_y = 0.000172

calibrant_type = "CeO2"
num_calibration_rings = 15

output_calibration_path = "../../../SXRD_analysis/diamond_2021/103852-calibration/calibration-pyFAI/calibration_103852_CeO2_737mm_pyFAI.poni"

In [ ]:
# Diamond 2021
input_calibrant_file = "../../../SXRD_raw_data/diamond_2021/rawdata_tiffs/103853_tiffs/00001_00000.tiff"

beam_centre_x = 755.767 # x-coordinate of the beam-center in pixels
beam_centre_y = 1679 - 998.124 # y-coordinate of the beam-center in pixels
sample_detector_distance = 907.3740 # This is the distance in mm (unit used by Fit2d)
wl = 1.242000e-11 # Beam wavelength

pixel_x = 0.000172
pixel_y = 0.000172

calibrant_type = "CeO2"
num_calibration_rings = 15

output_calibration_path = "../../../SXRD_analysis/diamond_2021/103853-calibration/calibration-pyFAI/calibration_103853_CeO2_907mm_pyFAI.poni"

In [ ]:
calibrate(input_calibrant_file, beam_centre_x, beam_centre_y, sample_detector_distance, wl, pixel_x, pixel_y, 
          calibrant_type, num_calibration_rings, output_calibration_path)

# Load calibration

In [ ]:
ai = pyFAI.load(output_calibration_path)
print("\nIntegrator: \n", ai)

## Create mask

In [ ]:
image = fabio.open(input_calibrant_file)
pattern_image_array = image.data
print("pattern_image_array:", type(pattern_image_array), pattern_image_array.shape, pattern_image_array.dtype)
jupyter.display(pattern_image_array)

mask = pattern_image_array < 0
print("mask:", type(mask), mask.shape, mask.dtype)
print("percent of bad pixels : ", 100*mask.sum()/(mask.shape[0]*mask.shape[1]))
jupyter.display(mask)

# Azimuthal integration of multiple images for `TOPAS`

In [ ]:
def azimuthal_integration_iteration(input_path: str, input_experiment_list: list, 
                                    output_path: str, output_experiment_list: list,
                                    ai, mask, number_of_points: int):

    # suppress warnings when TIFFs are read
    logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

    for input_experiment, output_experiment in zip(input_experiment_list, output_experiment_list):

        # get a list of the files
        image_list = sorted(pathlib.Path(input_path + input_experiment).glob("Ti64*.tif"))

        for image_path in tqdm(image_list):
            # create an image array and integrate the data
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                image = fabio.open(image_path)
            pattern_image_array = image.data
            result = ai.integrate1d(pattern_image_array,
                                number_of_points,
                                azimuth_range=(-180,180),
                                mask = mask,
                                unit="2th_deg",
                                correctSolidAngle=True,
                                polarization_factor=0.99,
                                method='full_csr')

            result_array = np.column_stack((result.radial, result.intensity))
            
            # check output folder exists
            output_folder = f"{output_path}{output_experiment}/azimuthal-integration/"
            CHECK_FOLDER = os.path.isdir(output_folder)

            if not CHECK_FOLDER:
                os.makedirs(output_folder)
                print("Created folder : ", output_folder)
            
            # write out the integrated data to a text file
            np.savetxt(f"{output_folder}{image_path.stem}.xy", result_array)
            
        print(f"Saved .xy azimuthal integration files to folder : {output_folder}")

In [ ]:
input_path = "../../../SXRD_raw_data/desy_2021/diffraction_images/"
input_experiment_list = ["Def_01","Def_03","Def_04","Def_05","Def_06","Def_07","Def_08","Def_09","Def_10","Heat_02"]

output_path = "../../../SXRD_analysis/desy_2021/"
output_experiment_list = ["experiment01-deformation",
                          "experiment03-deformation",
                          "experiment04-deformation",
                          "experiment05-deformation",
                          "experiment06-deformation",
                          "experiment07-deformation",
                          "experiment08-deformation",
                          "experiment09-deformation",
                          "experiment10-deformation",
                          "experiment02-heating"]

number_of_points = 10000

In [ ]:
azimuthal_integration_iteration(input_path, input_experiment_list, output_path, output_experiment_list, 
                                ai, mask, number_of_points)

# Caking multiple images for `xrdfit`

In [ ]:
def caking_iteration_xrdfit(input_path: str, input_experiment_list: list, 
                            output_path: str, output_experiment_list: list,
                            ai, mask, number_of_points: int, number_of_cakes: int):
    
    # suppress warnings when TIFFs are read
    logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

    # rotate the detector by half a cake width so that the cardinal direction
    # is in the center of the first cake not the edge.
    first_cake_angle = 360 / number_of_cakes
    ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians

    for input_experiment, output_experiment in zip(input_experiment_list, output_experiment_list):
        
        # get a list of the files
        image_list = sorted(pathlib.Path(input_path + input_experiment).glob("Ti64*.tif"))

        for image_path in tqdm(image_list):
            # create empty array
            caked_data = np.zeros((number_of_cakes + 1, number_of_points))

            # create an image array and cake the data
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                image = fabio.open(image_path)
            pattern_image_array = image.data
            result2d = ai.integrate2d(pattern_image_array,
                                    number_of_points,
                                    number_of_cakes,
                                    mask = mask,
                                    unit="2th_deg",
                                    polarization_factor=0.99,
                                    method='full_csr')

            # flip the intensity data to order cakes clockwise rather than anticlockwise
            intensity = np.flip(result2d.intensity.T, axis=1)

            # reshape radial labels to 2D array so they can be attached to the intensity data.
            radial = np.reshape(result2d.radial, (-1, 1))

            result_array = np.hstack((radial, intensity))

            # check output folder exists
            output_folder = f"{output_path}{output_experiment}/caking/"
            CHECK_FOLDER = os.path.isdir(output_folder)

            if not CHECK_FOLDER:
                os.makedirs(output_folder)
                print("Created folder : ", output_folder)

            # write out the caked data to a text file
            np.savetxt(f"{output_folder}{image_path.stem}.dat", result_array)

        print(f"Saved .dat caked data files to folder : {output_folder}")

In [ ]:
number_of_cakes = 72

In [ ]:
caking_iteration_xrdfit(input_path, input_experiment_list, output_path, output_experiment_list, 
                        ai, mask, number_of_points, number_of_cakes)

## Caking multiple images for `MAUD`

In [ ]:
def caking_iteration_maud(input_path: str, input_experiment_list: list, 
                          output_path: str, output_experiment_list: list,
                          ai, mask, pixel_size: float, number_of_points: int, number_of_cakes: int):
    
    # suppress warnings when TIFFs are read
    logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

    # rotate the detector by half a cake width so that the cardinal direction
    # is in the center of the first cake not the edge.
    first_cake_angle = 360 / number_of_cakes
    ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians

    for input_experiment, output_experiment in zip(input_experiment_list, output_experiment_list):
    
        # get a list of the files
        # image_list = sorted(pathlib.Path(input_path + input_experiment).glob("Ti64*.tif"))
        # image_list = sorted(pathlib.Path(input_path + input_experiment).glob("1*.tiff"))
        image_list = sorted(pathlib.Path(input_path + input_experiment).glob("*.cbf"))
        
        for image_path in tqdm(image_list):
            # create empty array
            caked_data = np.zeros((number_of_cakes + 1, number_of_points))

            # create an image array and cake the data
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                image = fabio.open(image_path)
            pattern_image_array = image.data
            result2d = ai.integrate2d(pattern_image_array,
                                    number_of_points,
                                    number_of_cakes,
                                    mask = mask,
                                    unit="r_mm",
                                    polarization_factor=0.99,
                                    method='full_csr')

            # flip the intensity data to order cakes clockwise rather than anticlockwise
            intensity = np.flip(result2d.intensity.T, axis=1)

            # reshape radial labels to 2D array so they can be attached to the intensity data.
            radial_mm = np.reshape(result2d.radial, (-1, 1))
            radial_pixel = radial_mm/pixel_size

            result_array = np.hstack((radial_pixel, intensity))
            
            # check output folder exists
            output_folder = f"{output_path}{output_experiment}/texture-maud/"
            CHECK_FOLDER = os.path.isdir(output_folder)

            if not CHECK_FOLDER:
                os.makedirs(output_folder)
                print("Created folder : ", output_folder)
            
            # write out the caked data to a text file
            np.savetxt(f"{output_folder}{image_path.stem}.dat", result_array)
        
        print(f"Saved .dat caked data files to folder : {output_folder}")     

In [ ]:
#pixel_size = 0.296 # Diamond 2017
#pixel_size = 0.200 # DESY 2021
pixel_size = 0.172 # Diamond 2021

In [ ]:
input_path = "../../../SXRD_analysis/diamond_2021/"
input_experiment_list = [
                         "103828/texture-cpf",
                         "103829/texture-cpf",
                         "103830/texture-cpf",
                         "103831/texture-cpf",
                         "103832/texture-cpf",
                         "103833/texture-cpf",
                         "103834/texture-cpf",
                         "103835/texture-cpf",
                         "103836/texture-cpf",
                         "103837/texture-cpf",
                         "103838/texture-cpf",
                         "103839/texture-cpf",
#                          "103840/texture-cpf",
#                          "103841/texture-cpf",
#                          "103842/texture-cpf",
#                          "103843/texture-cpf",
#                          "103844/texture-cpf",
#                          "103845/texture-cpf",
#                          "103846/texture-cpf",
#                          "103847/texture-cpf",
#                          "103848/texture-cpf",
#                          "103849/texture-cpf",
#                          "103850/texture-cpf",
#                          "103851/texture-cpf"
                        ]

output_path = input_path
output_experiment_list = [
                         "103828",
                         "103829",
                         "103830",
                         "103831",
                         "103832",
                         "103833",
                         "103834",
                         "103835",
                         "103836",
                         "103837",
                         "103838",
                         "103839",
#                          "103840",
#                          "103841",
#                          "103842",
#                          "103843",
#                          "103844",
#                          "103845",
#                          "103846",
#                          "103847",
#                          "103848",
#                          "103849",
#                          "103850",
#                          "103851"
                         ]

number_of_points = 10000

In [ ]:
input_path = "../../../SXRD_analysis/diamond_2021/"
input_experiment_list = ["103853-calibration"]

output_path = input_path
output_experiment_list = ["103853-calibration"]

number_of_points = 10000

In [ ]:
input_path = "../../../SXRD_raw_data/diamond_2021/rawdata/"
input_experiment_list = ["103845-pilatus2M-files"]

output_path = "../../../SXRD_analysis/diamond_2021/"
output_experiment_list = ["103845"]

number_of_points = 10000

In [ ]:
caking_iteration_maud(input_path, input_experiment_list, output_path, output_experiment_list, 
                      ai, mask, pixel_size, number_of_points, number_of_cakes)